In [1]:
import json

import numpy as np
from paprika.analysis import get_block_sem, get_nearest_max
from paprika.io import json_numpy_obj_hook

In [2]:
def block_sem(data):
    avgs = np.mean(data)
    totn = data.size
    blkn = get_nearest_max(totn)
    sems = get_block_sem(data[:blkn])

    return [avgs, sems]

### Load Data

In [3]:
with open("potential_decomposition-a000.json", "r") as f:
    bound = json.load(f, object_hook=json_numpy_obj_hook)

In [4]:
with open("potential_decomposition-r014.json", "r") as f:
    unbound = json.load(f, object_hook=json_numpy_obj_hook)

In [5]:
bound_potential = (
    bound["bond"]
    + bound["angle"]
    + bound["dihedral"]
    + bound["Elec"]
    + bound["Elec14"]
    + bound["LJ"]
    + bound["LJ14"]
)
unbound_potential = (
    unbound["bond"]
    + unbound["angle"]
    + unbound["dihedral"]
    + unbound["Elec"]
    + unbound["Elec14"]
    + unbound["LJ"]
    + unbound["LJ14"]
)
enthalpy = {key: np.mean(bound[key]) - np.mean(unbound[key]) for key in bound}

### Valence term

In [6]:
valence = enthalpy["bond"] + enthalpy["angle"] + enthalpy["dihedral"]
valence

-2.0732016404146165

### Lennard-Jones term

In [7]:
LJ = enthalpy["LJ"] + enthalpy["LJ14"]
LJ

-14.378978995981672

### Electrostatic term

In [8]:
Elec = enthalpy["Elec"] + enthalpy["Elec14"]
Elec

-9.42081003812973

### Sum of all components

In [9]:
potential = LJ + Elec + valence
potential

-25.87299067452602

### Total Enthalpy

In [10]:
bound = block_sem(bound_potential)
unbound = block_sem(unbound_potential)
enthalpy = {
    "pe": bound[0] - unbound[0],
    "sem": (bound[1] ** 2 + unbound[1] ** 2) ** 0.5,
}
print(f"Bound Potential  : {bound[0]:.2f} +- {bound[1]:.2f} kcal/mol")
print(f"Unbound Potential: {unbound[0]:.2f} +- {unbound[1]:.2f} kcal/mol")
print(f"Binding Enthalpy : {enthalpy['pe']:.2f} +- {enthalpy['sem']:.2f} kcal/mol")

Bound Potential  : -21865.44 +- 0.17 kcal/mol
Unbound Potential: -21839.57 +- 0.22 kcal/mol
Binding Enthalpy : -25.87 +- 0.27 kcal/mol
